# Notebook 03: Entrenamiento y Optimización (OE3)

## 📊 Análisis y Corrección de Resultados Fallidos (Diagnóstico)
Tras analizar las corridas previas en Colab, identificamos el problema crítico:
- **Error:** `loss: nan` y accuracy estancada en ~20-30%.
- **Diagnóstico:** Los modelos pre-entrenados (MobileNetV3) esperan valores de entrada entre `[-1, 1]`, pero recibían imágenes crudas `[0, 255]`. Esto causó inestabilidad numérica.
- **Resultados Previos:** El modelo no aprendió (Underfitting severo).

## 🚀 Solución Implementada: Estrategia Robusta
Para lograr un **Accuracy Óptimo (>60%)**, hemos actualizado el código con:
1.  **Normalización Correcta:** Capa `Rescaling(1./127.5, offset=-1)` integrada.
2.  **Data Augmentation:** Rotación, zoom y flip integrados para evitar memorización.
3.  **Fine-Tuning (Afinamiento):**
    - Fase 1: Entrenar solo el clasificador final (20 épocas).
    - Fase 2: Descongelar capas superiores y re-entrenar con learning rate bajo (10 épocas extra).

Esta estrategia aprovecha el conocimiento previo de MobileNetV3 (transfer learning) para adaptarse a las emociones.

In [ ]:
!pip install tensorflow numpy matplotlib

In [ ]:
import os
import sys
import tensorflow as tf

# Configuración Híbrida (Drive vs Local)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    # AJUSTA ESTA RUTA A TU CARPETA EN DRIVE si es necesario
    base_path = '/content/drive/MyDrive/Cursos/Posgrado-FIIS/MIA/MIA-403-A-Tesis2/Tesis_EdgeAI/Proy_Repo'
    # base_path = '/content/drive/MyDrive/Tesis_EdgeAI/Proy_Repo' # Ruta alternativa
    if not os.path.exists(base_path):
         print(f"Ruta no encontrada: {base_path}. Intentando ruta alternativa estándar...")
         base_path = '/content/drive/MyDrive/Tesis_EdgeAI/Proy_Repo'
    
    if os.path.exists(base_path):
         os.chdir(base_path)
         print(f"Ejecutando en COLAB desde: {base_path}")
    else:
         print("⚠️ NO SE ENCONTRÓ LA CARPETA DEL PROYECTO EN DRIVE. Verifica la ruta.")
except ImportError:
    print("Ejecutando en LOCAL")

sys.path.append(os.path.abspath('.'))

In [ ]:
from src.models.architectures import build_mini_xception, build_mobilenet_v3_small
from src.models.train import train_model
from src.models.optimize import ModelOptimizer

In [ ]:
print("GPUs Disponibles:", tf.config.list_physical_devices('GPU'))

## 2. Entrenamiento Optimizado (Fase 1 + Fase 2 Auto)
Ejecuta la siguiente celda. Ahora el script `train_model` se encargará automáticamente del Fine-Tuning.

In [ ]:
# Entrenar MobileNetV3 con la nueva estrategia
# Se ejecutará por 20 épocas (Fase 1) + 10 épocas (Fine Tuning)
history = train_model("data/raw/fer2013", model_type="mobilenet", epochs=20, batch_size=32)
model_to_save = "mobilenet"

## 3. Conversión a TFLite (Edge Ready)
El paso final para la tesis: generar el archivo optimizado para Raspberry Pi / Jetson.

In [ ]:
model_path = f"models/checkpoints/{model_to_save}_final.keras"

if os.path.exists(model_path):
    optimizer = ModelOptimizer(model_path)
    optimizer.convert_to_tflite(quantization='int8')
    print(f"Modelo optimizado guardado en models/optimized/")
else:
    print(f"No se encontró {model_path}. Asegúrate de que el entrenamiento terminó bien.")